In [34]:
import pathlib
import chess
import chess.pgn
import chess.engine
import numpy as np
import h5py

In [35]:
pgnGames = list(pathlib.Path('test_pgns').glob('*.pgn'))

In [36]:
# General space saving algorithms that are often used in chess programming sites

def board_to_arr(board: chess.Board):
    black, white = board.occupied_co

    bitboards = np.array([
        black & board.pawns,
        black & board.knights,
        black & board.bishops,
        black & board.rooks,
        black & board.queens,
        black & board.kings,
        white & board.pawns,
        white & board.knights,
        white & board.bishops,
        white & board.rooks,
        white & board.queens,
        white & board.kings,
    ], dtype=np.uint64)

    return bitboards

def bitboards_to_array(bb: np.ndarray) -> np.ndarray:
    bb = np.asarray(bb, dtype=np.uint64)[:, np.newaxis]
    s = 8 * np.arange(7, -1, -1, dtype=np.uint64)
    b = (bb >> s).astype(np.uint8)
    b = np.unpackbits(b, bitorder="little")
    return b.reshape(-1, 8, 8)


In [37]:
engine = chess.engine.SimpleEngine.popen_uci("engines/stockfish-macos-x86-64-bmi2")

def convert_to_winrate(acpl):
    
    return (50 + 50 * (2 / (1 + np.exp(-0.00368208 * acpl)) - 1)) * 0.01

def evaluate_game(game, think_time=0.1, verbose=False):
    
    single_game = []
    evaluations = []
    
    board = game.board()

    for move in game.mainline_moves():
    
        # Evaluate the current position 
        result = engine.analyse(board, chess.engine.Limit(time=think_time))
        
        if result['score'].turn: 
            try:
                objective_eval = result['score'].relative.cp
                evaluations.append(convert_to_winrate(objective_eval))
            except:
                if result['score'].relative.moves > 0:
                    evaluations.append(1)
                else:
                    evaluations.append(0)
        else:
            try:
                objective_eval = result['score'].relative.cp * -1
                evaluations.append(convert_to_winrate(objective_eval))
            except:
                if result['score'].relative.moves < 0:
                    evaluations.append(1)
                else:
                    evaluations.append(0)       
        
        # Push the game state to new
        board.push_uci(move.uci())
        single_game.append(board_to_arr(board))
    
    # Evaluate final position once more to find accuracy of last move
    result = engine.analyse(board, chess.engine.Limit(time=think_time))
    evaluations.append(convert_to_winrate(objective_eval))
    if verbose:
        print(evaluations)
    
    winrate_diff = [abs(x - evaluations[i - 1]) for i, x in enumerate(evaluations)][1:]

    return single_game, winrate_diff
    

In [38]:
X = []
y = []

for g in range(len(pgnGames)):
    pgn = open(pgnGames[g])
    
    for k in range(500):  # 190,000 assures all games are looked at.
        
        try:
            game = chess.pgn.read_game(pgn)

            board_history, winrate_history = evaluate_game(game, think_time=0.025, verbose=False)
            X += board_history
            y += winrate_history
            print(f"{k+1} games analyzed, dataset size {len(X)}")
        except Exception as e:
            print(f"{e}")
            pass
          

1 games analyzed, dataset size 31
2 games analyzed, dataset size 41
local variable 'objective_eval' referenced before assignment
4 games analyzed, dataset size 61
5 games analyzed, dataset size 63
6 games analyzed, dataset size 118
7 games analyzed, dataset size 199
8 games analyzed, dataset size 280
9 games analyzed, dataset size 314
10 games analyzed, dataset size 365
11 games analyzed, dataset size 431
12 games analyzed, dataset size 484
13 games analyzed, dataset size 529
14 games analyzed, dataset size 594
15 games analyzed, dataset size 637
16 games analyzed, dataset size 760
17 games analyzed, dataset size 886
18 games analyzed, dataset size 957
19 games analyzed, dataset size 1076
20 games analyzed, dataset size 1146
21 games analyzed, dataset size 1174
22 games analyzed, dataset size 1209
23 games analyzed, dataset size 1250
24 games analyzed, dataset size 1290
25 games analyzed, dataset size 1326
26 games analyzed, dataset size 1371
27 games analyzed, dataset size 1436
28 gam

217 games analyzed, dataset size 14727
218 games analyzed, dataset size 14785
219 games analyzed, dataset size 14894
220 games analyzed, dataset size 14940
221 games analyzed, dataset size 15056
222 games analyzed, dataset size 15123
223 games analyzed, dataset size 15162
224 games analyzed, dataset size 15271
225 games analyzed, dataset size 15333
226 games analyzed, dataset size 15405
227 games analyzed, dataset size 15489
228 games analyzed, dataset size 15613
229 games analyzed, dataset size 15734
230 games analyzed, dataset size 15772
231 games analyzed, dataset size 15824
232 games analyzed, dataset size 15883
233 games analyzed, dataset size 15939
234 games analyzed, dataset size 16001
235 games analyzed, dataset size 16028
236 games analyzed, dataset size 16080
237 games analyzed, dataset size 16145
238 games analyzed, dataset size 16209
239 games analyzed, dataset size 16253
240 games analyzed, dataset size 16311
local variable 'objective_eval' referenced before assignment
242

425 games analyzed, dataset size 29628
426 games analyzed, dataset size 29669
427 games analyzed, dataset size 29708
428 games analyzed, dataset size 29806
429 games analyzed, dataset size 29884
430 games analyzed, dataset size 29944
431 games analyzed, dataset size 30063
432 games analyzed, dataset size 30138
433 games analyzed, dataset size 30225
434 games analyzed, dataset size 30341
435 games analyzed, dataset size 30389
436 games analyzed, dataset size 30462
437 games analyzed, dataset size 30521
438 games analyzed, dataset size 30596
439 games analyzed, dataset size 30701
440 games analyzed, dataset size 30764
441 games analyzed, dataset size 30862
442 games analyzed, dataset size 30899
443 games analyzed, dataset size 30974
444 games analyzed, dataset size 31037
445 games analyzed, dataset size 31159
446 games analyzed, dataset size 31224
447 games analyzed, dataset size 31247
448 games analyzed, dataset size 31318
449 games analyzed, dataset size 31369
450 games analyzed, datas

In [39]:
X = np.dstack(X)
y = np.array(y, dtype='float32')
X = np.rollaxis(X, -1).reshape(-1, 12)

In [40]:
with h5py.File('test_positions.h5', 'w') as hf:
    hf.create_dataset("X", data=X, compression='gzip', compression_opts=5)
    hf.create_dataset("y", data=y, compression='gzip', compression_opts=5)

## Try Model on Test Dataset

In [41]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_kernels, v_planes=2):
        super(ResNet, self).__init__()
        self.in_planes = 12
        self.value_planes = v_planes

        self.conv1 = nn.Conv2d(self.in_planes, self.in_planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(self.in_planes)
        self.layer1 = self._make_layer(block, num_kernels[0], num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, num_kernels[1], num_blocks[1], stride=1)
        self.layer3 = self._make_layer(block, num_kernels[2], num_blocks[2], stride=1)
        self.layer4 = self._make_layer(block, num_kernels[3], num_blocks[3], stride=1)

        # value head
        self.finalLayerValue = nn.Sequential(
            nn.Conv2d(num_kernels[3], self.value_planes, kernel_size=1, stride=1, padding=0),  # 64, 1
            nn.BatchNorm2d(self.value_planes),
            nn.ReLU()
            )
        self.valueLinear = nn.Linear(64*self.value_planes*block.expansion, 1)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        
        out = self.finalLayerValue(out)
        out = out.view(out.size(0), -1)
        out = self.valueLinear(out)
        return out


In [71]:
def ResNetSmall():
    return ResNet(BasicBlock, [1,1,1,2], [32,32,32,32], v_planes=2)

def ResNetRegular():
    return ResNet(BasicBlock, [3,3,3,3], [256,256,256,256], v_planes=16)

In [72]:
class TrainingDataset(torch.utils.data.Dataset):

    def __init__(self, X, y):
        self.X = X
        self.y = torch.from_numpy(y)

    def __getitem__(self, index):

        return torch.from_numpy(bitboards_to_array(self.X[index])).float(), self.y[index]

    def __len__(self):
        return len(self.X)

In [79]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ResNetRegular()
model.load_state_dict(torch.load('vol_model_updated.pt', map_location=torch.device(device)))
model

ResNet(
  (conv1): Conv2d(12, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(12, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential(
        (0): Conv2d(12, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): BasicBlock(
      (conv1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=

In [80]:
testloader = torch.utils.data.DataLoader(dataset=TrainingDataset(X, y), batch_size=64, shuffle=False)

In [81]:
# Define a criterion for calculating the MSE
criterion = nn.MSELoss()

# Set the model to evaluation mode
model.eval()

# Initialize variables to store total loss and total number of samples
total_loss = 0.0
total_samples = 0

losses = []

# Iterate over the test dataset
i = 0
for inputs, labels in testloader:
    # Move inputs and labels to the device (e.g., GPU) if available
    inputs = inputs.to(device)
    labels = labels.to(device)

    # Perform forward pass
    with torch.no_grad():
        outputs = model(inputs)
    
    # Calculate the loss
    loss = criterion(outputs, labels)

    # Update total loss and total number of samples
    total_loss += loss.item() * inputs.size(0)
    total_samples += inputs.size(0)
    losses += list(outputs.flatten().detach().cpu().numpy())
    if i % 25 == 0:
        print(f"After {i+1} batches, {len(losses)} samples: {total_loss/total_samples}")
    i += 1

# Calculate the average MSE
mse = total_loss / total_samples

print("Mean Squared Error on Test Dataset:", mse)

After 1 batches, 64 samples: 0.004126541782170534
After 26 batches, 1664 samples: 0.006925160204096196
After 51 batches, 3264 samples: 0.006851481623016298
After 76 batches, 4864 samples: 0.0071175210522549034
After 101 batches, 6464 samples: 0.007258291515328064
After 126 batches, 8064 samples: 0.007254029524841508
After 151 batches, 9664 samples: 0.0073785725980997086
After 176 batches, 11264 samples: 0.008118359542674047
After 201 batches, 12864 samples: 0.007749638081741385
After 226 batches, 14464 samples: 0.007734240359313522
After 251 batches, 16064 samples: 0.0073865170377692526
After 276 batches, 17664 samples: 0.0076545180689182426
After 301 batches, 19264 samples: 0.007526241586716889
After 326 batches, 20864 samples: 0.007371178510582055
After 351 batches, 22464 samples: 0.007434416805970101
After 376 batches, 24064 samples: 0.007366359527564937
After 401 batches, 25664 samples: 0.007339489261410432
After 426 batches, 27264 samples: 0.007242688191244368
After 451 batches, 2

In [83]:
max(losses)

0.13946086

In [84]:
min(losses)

-0.30245227

In [86]:
sorted(losses)[::-1]

[0.13946086,
 0.1293797,
 0.12164103,
 0.11902509,
 0.117413275,
 0.11166188,
 0.110739596,
 0.107979186,
 0.10600522,
 0.10273534,
 0.102454446,
 0.102325834,
 0.10078408,
 0.100346096,
 0.0980807,
 0.09745741,
 0.09745294,
 0.09677606,
 0.096512996,
 0.09626369,
 0.095383435,
 0.095316276,
 0.09464004,
 0.09426595,
 0.09411624,
 0.09321528,
 0.0929867,
 0.092877164,
 0.09277512,
 0.09271425,
 0.09242788,
 0.0917902,
 0.0916804,
 0.091618024,
 0.09129824,
 0.09120181,
 0.08994926,
 0.08944433,
 0.089137584,
 0.088953055,
 0.08889559,
 0.08838857,
 0.08779136,
 0.08776175,
 0.08757596,
 0.087187886,
 0.08711198,
 0.086721696,
 0.08671602,
 0.08627905,
 0.08627735,
 0.085768506,
 0.08571564,
 0.08567254,
 0.08559756,
 0.08546743,
 0.08528513,
 0.08496798,
 0.084723786,
 0.08411734,
 0.08369338,
 0.083686866,
 0.08351062,
 0.08351062,
 0.08346775,
 0.08342728,
 0.083420694,
 0.083138086,
 0.08294839,
 0.08257344,
 0.082295276,
 0.0822784,
 0.08220789,
 0.0822026,
 0.08189574,
 0.08177704